In [1]:
import numpy as np
from qiskit.opflow import *
from qiskit.aqua.operators import SummedOp, PauliOp, MatrixOp
from qiskit.quantum_info import *

In [2]:
N = 5
num_turns = 2*(N - 1)
side_chain = [0]*N
lambda_back = 10

# GLOBAL
FULL_ID = I
for i in range(1, num_turns):
    FULL_ID = I^FULL_ID

In [3]:
def _create_pauli_for_conf(N):
    terms = []
    num_turns = 2*(N - 1)
    pauli_conf = np.zeros((num_turns, 2), dtype=object)
    
    for index in range(num_turns):
        if index != 0: 
            temp = I
        else: 
            temp = Z
        for i in range(1, num_turns):
            if i == index: 
                temp = Z^temp
            else:
                temp = I^temp
        terms.append(temp)    
        
    for i in range(num_turns):
        pauli_conf[i][0] = terms[i]
        pauli_conf[i][1] = terms[i]   
    return pauli_conf

def _create_qubits_for_conf(pauli_conf):
    qubits = np.zeros(pauli_conf.shape, dtype=object)
    num_turns = qubits.shape[0]
    for i in range(num_turns):
        qubits[i][0] = (0.5*FULL_ID - 0.5*pauli_conf[i][0])
        qubits[i][1] = (0.5*FULL_ID - 0.5*pauli_conf[i][1])
    return qubits

# Create paulis for conformation

In [4]:
pauli_conf = _create_pauli_for_conf(N)

In [5]:
pauli_conf

array([[PauliOp(Pauli('IIIIIIIZ'), coeff=1.0),
        PauliOp(Pauli('IIIIIIIZ'), coeff=1.0)],
       [PauliOp(Pauli('IIIIIIZI'), coeff=1.0),
        PauliOp(Pauli('IIIIIIZI'), coeff=1.0)],
       [PauliOp(Pauli('IIIIIZII'), coeff=1.0),
        PauliOp(Pauli('IIIIIZII'), coeff=1.0)],
       [PauliOp(Pauli('IIIIZIII'), coeff=1.0),
        PauliOp(Pauli('IIIIZIII'), coeff=1.0)],
       [PauliOp(Pauli('IIIZIIII'), coeff=1.0),
        PauliOp(Pauli('IIIZIIII'), coeff=1.0)],
       [PauliOp(Pauli('IIZIIIII'), coeff=1.0),
        PauliOp(Pauli('IIZIIIII'), coeff=1.0)],
       [PauliOp(Pauli('IZIIIIII'), coeff=1.0),
        PauliOp(Pauli('IZIIIIII'), coeff=1.0)],
       [PauliOp(Pauli('ZIIIIIII'), coeff=1.0),
        PauliOp(Pauli('ZIIIIIII'), coeff=1.0)]], dtype=object)

# Create qubits for conformation

In [6]:
qubits = _create_qubits_for_conf(pauli_conf)

In [23]:
def _create_indic_turn(N, side_chain, qubits):
    if len(side_chain)!= N:
        raise Exception('size of side_chain list is not equal to N')
    num_turns = N - 1 
    indic_0 = np.zeros((num_turns, 2), dtype=object)
    indic_1 = np.zeros((num_turns, 2), dtype=object)
    indic_2 = np.zeros((num_turns, 2), dtype=object)
    indic_3 = np.zeros((num_turns, 2), dtype=object)
    r_conf = 0
    for i in range(num_turns):
        for m in range(2):
            if m == 1:
                if side_chain[i - 1] == 0:
                    continue
                else:
                    pass
            indic_0[i][m] = (FULL_ID - qubits[2*i][m])@(FULL_ID - qubits[2*i + 1][m])
            indic_1[i][m] = qubits[2*i + 1][m]@(qubits[2*i + 1][m] - 1*qubits[2*i][m])
            indic_2[i][m] = qubits[2*i][m]@(qubits[2*i][m] -1*qubits[2*i + 1][m])
            indic_3[i][m] = qubits[2*i][m]@(qubits[2*i + 1][m])
            r_conf += 1
    num_qubits = 2*r_conf - 5
    print('number of qubits required for conformation: ', num_qubits)
    return indic_0, indic_1, indic_2, indic_3, num_qubits


def _check_turns(i, p, j, s,
                 indic0, indic1, indic2, indic3):
    t_ij = indic0[i][p]@indic0[j][s] + indic1[i][p]@indic1[j][s] + \
           indic2[i][p]@indic2[j][s] + indic3[i][p]@indic3[j][s] 
    return t_ij


def _create_H_back(N, lambda_back, indic_0,
                   indic_1, indic_2, indic_3):
    H_back = 0
    for i in range(N - 2):
        H_back += lambda_back*_check_turns(i, 0, i + 1, 0,
                                           indic_0, indic_1, indic_2, indic_3)
    H_back = H_back.reduce()
    return H_back

In [24]:
indic_0, indic_1, indic_2, indic_3, num_qubits = _create_indic_turn(N, side_chain, qubits)
H_back = _create_H_back(N, lambda_back, indic_0, indic_1, indic_2, indic_3)

number of qubits required for conformation:  3


## Need to create a function that enforces pre-set binaries

As in the paper, first two turns are fixed to be (in binary)
01 and 00, which translate to 1,-1 and 1,1, respectively in
spin variables. With no side chain on the 2nd bead, we further
fix the value of the 6th qubit, q_6 = 1 (-1 for the corresponding
Pauli).

So, we change:
- Pauli[0] = I
- Pauli[1] = -I
- Pauli[2] = I
- Pauli[3] = I
- Pauli[5] = -I

In [20]:
def _set_binaries(H_back):
    new_tables = []
    new_coeffs = []
    for i in range(len(H_back)):
        H = H_back[i]
        table_Z = np.copy(H.primitive.table.Z[0])
        table_X = np.copy(H.primitive.table.X[0])
        # get coeffs and update 
        coeffs = np.copy(H.primitive.coeffs[0])
        if table_Z[1] == np.bool_(True):
            coeffs = -1*coeffs
        if table_Z[5] == np.bool_(True):
            coeffs = -1*coeffs
        # impose preset binary values
        table_Z[0] = np.bool_(False)
        table_Z[1] = np.bool_(False)
        table_Z[2] = np.bool_(False)
        table_Z[3] = np.bool_(False)
        table_Z[5] = np.bool_(False)        
        new_table = np.concatenate((table_X, table_Z), axis=0)
        new_tables.append(new_table)
        new_coeffs.append(coeffs)
    new_pauli_table = PauliTable(data=new_tables)
    H_back_updated = PauliSumOp(SparsePauliOp(data=new_pauli_table, coeffs=new_coeffs))     
    H_back_updated = H_back_updated.reduce()
    return H_back_updated

In [14]:
H_back_updated = _set_binaries(H_back)

In [15]:
for H in H_back_updated:
    print(H)

2.5 * IIIIIIII
-2.5 * ZIIIIIII
2.5 * IZIZIIII
-2.5 * ZZIZIIII


## Need to create a 'mask' that will create the qubit operator with the right number of qubits (in this case, 3)

Or possible combine setting binaries and change number of qubits in one function/method?

## Compare to latticefolding problem implementation using symbolic mathematics

In [18]:
from qiskit_nature.problems.sampling.folding import LatticeFoldingProblem
# default parameters are the same: 5 letter peptide 
lf = LatticeFoldingProblem()
lf.pauli_op()
# final result is the SummedOp

number of qubits required for conformation:  3
10  distances created
number of qubits required for contact :  0
total number of qubits required : 3
number of terms in the hamiltonian :  4
Hamiltonian:  -5*\sigma^z_{5}*\sigma^z_{7}*\sigma^z_{8}/2 + 5*\sigma^z_{5}*\sigma^z_{7}/2 - 5*\sigma^z_{8}/2 + 5/2
new qubits:  [0, \sigma^z_{5}, \sigma^z_{7}, \sigma^z_{8}]
Mask is [[ 2.5  0.   0.   0. ]
 [-2.5  0.   0.   1. ]
 [ 2.5  1.   1.   0. ]
 [-2.5  1.   1.   1. ]]
pauli_list:  [array([-2.5, Pauli('ZZZ')], dtype=object), array([2.5, Pauli('IZZ')], dtype=object), array([-2.5, Pauli('ZII')], dtype=object), array([2.5, Pauli('III')], dtype=object)]


SummedOp([PauliOp(Pauli('ZZZ'), coeff=-2.5), PauliOp(Pauli('IZZ'), coeff=2.5), PauliOp(Pauli('ZII'), coeff=-2.5), PauliOp(Pauli('III'), coeff=2.5)], coeff=1.0, abelian=False)